In [474]:
import os
import pickle
import argparse

import gensim
from gensim.models import word2vec

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import nltk
from nltk.tokenize import word_tokenize


from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import LSTM, Bidirectional
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.utils import to_categorical
#x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
from keras import backend as K
from keras.callbacks import EarlyStopping

from sklearn import preprocessing


In [476]:
def sliding_window(df,window_size,window_stride,input_features=None, output_features=None):
    allData = df[input_features].values.tolist()
    if output_features:
        outputData = [i[0] for i in df[output_features].values.tolist()]
    myArray = [[]]
    outputDataArray = [] 
    
    start=0
    for i in range(0,len(allData),window_stride):
        if i == 0:
            myArray = [allData[i:window_size+i]]
            if output_features != None:
                #outputDataArray.append(outputData[window_size+i][0])
                outputDataArray=[outputData[i+window_size-1]]
                #print(1,type(outputDataArray))
        else:
            myArray.append(allData[i:window_size+i])
            
            if window_size+i >= len(allData):
                if output_features != None:
                    #print(2,type(outputDataArray))
                    outputDataArray.append(outputData[len(allData)-1])
                break
            if output_features != None:
                    #outputDataArray.append(outputData[window_size+i][0])
                    #print(3,type(outputDataArray))
                    outputDataArray.append(outputData[i+window_size-1])

    if output_features == None:
            return (np.array(myArray))
    print(np.shape(np.array(myArray)))
    #return (np.array(myArray), outputDataArray)
    return (myArray, outputDataArray)

def mystratifiedOutputSampler(df, target):
    #a = {i:[] for i in df[target].unique()}
    xTIndexes = []
    minLen = None
    #print(a)
    for i in df[target].unique():
        #a[i] = df.index[df[target] != i].tolist()
        a = df.index[df[target] != i].tolist()
        print(i,len(a))
        if minLen == None or minLen > len(a):
            minLen = len(a)
    #for i in a.keys():
    print(minLen)
    for i in df[target].unique():
        if not xTIndexes:
                print(2)
                xTIndexes = list(np.random.choice(df.index[df[target] != i].tolist(), size=minLen, replace=False))
        else:
            #print(OnlyNeutal)
            xTIndexes = xTIndexes + list(np.random.choice(df.index[df[target] != i].tolist(), size=minLen, replace=False))

    return (xTIndexes)

In [625]:

def fileProcessor(csvFile,stockName):
    dateparse = lambda x: pd.datetime.strptime(x, '%d-%m-%Y')
    df = pd.read_csv(csvFile, parse_dates=['date'],date_parser=dateparse)
    #df = pd.read_csv("/datadrive/Sahil/code/GL/fewTrails/Datasets/GE.csv", parse_dates=['date'],date_parser=dateparse)


    df["date"]  = pd.to_datetime(df.date)
    #type(df["date"].iloc[0])
    df = df.sort_values(by="date")

    df['close_delta'] = 0
    df['close_direction'] = 0
    df['Stock_name'] = stockName
    for index in range(0,df.shape[0]):
        #print(index,df.iloc[index]['close'])
        if index == 0:
            df['close_delta'].iloc[index] =  (float(0))
            df['close_direction'].iloc[index] =  1
        #elif index <= df.shape[0]-1:
        else:
            df['close_delta'].iloc[index] = ((df['close'].iloc[index] - df['close'].iloc[index-1])/df['close'].iloc[index-1])*100
            if df['close_delta'].iloc[index] >= 1:
                df['close_direction'].iloc[index] = 2
            elif df['close_delta'].iloc[index] <= -1:
                df['close_direction'].iloc[index] = 0
            else:
                df['close_direction'].iloc[index] = 1
    return (df)
'''
fileCount = 0
import os
for root, dirs, files in os.walk("/datadrive/Sahil/code/GL/fewTrails/Datasets"):
    for file in files:
        if file.endswith(".csv"):
            print(os.path.join(root, file))
            if fileCount == 0:
                df = fileProcessor(os.path.join(root, file),file.split('.')[0])
            else:
                df = df.append(fileProcessor(os.path.join(root, file),file.split('.')[0]))
                
'''    
gCount=0
#df.apply(lambda x: featureTransform(x,df.shape[0]),axis=1)
#df.shape[0]
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')
csvFile = "/datadrive/Sahil/code/GL/fewTrails/Datasets/ALL_clean.csv"
df = pd.read_csv(csvFile, parse_dates=['date'],date_parser=dateparse)

In [628]:
window_size = 1
input_cols = ['curr_ratio','tot_debt_tot_equity', 'oper_profit_margin','asset_turn','ret_equity','sentiment']

input_cols_scale = ['curr_ratio','tot_debt_tot_equity', 'oper_profit_margin','asset_turn','ret_equity',]

df = df.dropna().reset_index(drop=True)
#df_all.reset_index(drop=True)
mydf =  pd.DataFrame(preprocessing.scale(df[input_cols_scale]),columns=input_cols_scale)
mydf['sentiment'] = df['sentiment']
mydf['close_direction'] = df['close_direction']
mydf['Stock_name'] = df['Stock_name']
train_indexes = mystratifiedOutputSampler(mydf,'close_direction')
mydf = mydf.iloc[train_indexes]

x_train,y_train=sliding_window(mydf,window_size,1,input_cols,['close_direction'])
x_train = np.array(x_train)
print(np.shape(x_train))
for index,stock in enumerate(mydf['Stock_name'].unique()):
    print(stock)
    if index == 0:
        x_train,y_train=sliding_window(mydf[mydf['Stock_name'] == stock],window_size,1,input_cols,['close_direction'])
    else:
        x_,y_=sliding_window(mydf[mydf['Stock_name'] == stock],window_size,1,input_cols,['close_direction'])
        print(type(x_train),type(x_),np.shape(x_train))
        x_train = x_train + x_
        y_train = y_train + y_
x_train = np.array(x_train)
#y_train = np.array(y_train)

1 2693
0 7446
2 7175
2693
2
(8079, 1, 6)
(8079, 1, 6)
MCD
(1098, 1, 6)
GE
(1166, 1, 6)
<class 'list'> <class 'list'> (1098, 1, 6)
BOEING
(1243, 1, 6)
<class 'list'> <class 'list'> (2264, 1, 6)
DISNEY
(1166, 1, 6)
<class 'list'> <class 'list'> (3507, 1, 6)
APPLE
(1065, 1, 6)
<class 'list'> <class 'list'> (4673, 1, 6)
MICROSOFT
(1208, 1, 6)
<class 'list'> <class 'list'> (5738, 1, 6)
IBM
(1133, 1, 6)
<class 'list'> <class 'list'> (6946, 1, 6)


In [631]:
y_train1 = np.array(y_train)
y_train1 = np.array(y_train1).reshape((-1, 1))
#y_train = to_categorical(y_train)

from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
#integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y_train = onehot_encoder.fit_transform(y_train1)

In [632]:
np.shape(y_train)[1]
#onehot_encoder.transform(y_train1[0]).tolist()[0]
#for i in onehot_encoder.transform(y_train1):
#    print(i)

3

In [633]:
from sklearn.model_selection import train_test_split

def getBalancedSamples(x_train,proportionBYNonNeutal=.7,neutalProportionToOther=.7):
    #proportionBYNonNeutal = .7
    #neutalProportionToOther = .7
    AllWithAnyOtherThanNeutal = []
    OnlyNeutal=[]
    #for index,i in enumerate(x_train[1:500,:,:]):
    for index,i in enumerate(x_train[:,:,:]):
        #print()
        #print(i)
        #print(i[:,5])
        if 1 in i[:,5] or 2 in i[:,5] or 4 in i[:,5] or 5 in i[:,5]  :
            #print("TRUE")
            if not AllWithAnyOtherThanNeutal:
                AllWithAnyOtherThanNeutal = [index]
            else:
                AllWithAnyOtherThanNeutal.append(index)
        else:
            if not OnlyNeutal:
                print(2)
                OnlyNeutal = [index]
            else:
                #print(OnlyNeutal)
                OnlyNeutal.append(index)
    print (len(AllWithAnyOtherThanNeutal))
    print (len(OnlyNeutal))
    #AllWithAnyOtherThanNeutal[np.random.randint(0,100,size=20)]
    proportion = .7
    size = int(round(proportion*len(AllWithAnyOtherThanNeutal),0))
    print(size)
    #xTIndexesPart1 = [AllWithAnyOtherThanNeutal[i] for i in list(np.random.randint(0,100,size=size))]
    #xTIndexesPart2 = [OnlyNeutal[i] for i in list(np.random.randint(0,100,size=int(round(size*.7)))) ]
    #print(len(xTIndexesPart1),len(xTIndexesPart2))
    #xTIndexes = AllWithAnyOtherThanNeutal + OnlyNeutal
    
    #X_train, X_test, y_train, y_test = train_test_split(x_train[xTIndexes,:,:], y[xTIndexes], test_size=0.2)
    xTIndexesPart1 = list(np.random.choice(AllWithAnyOtherThanNeutal, size=size, replace=False))
    xTIndexesPart2 = list(np.random.choice(OnlyNeutal, size=int(round(size*.7)), replace=False))
    xTIndexes = xTIndexesPart1 + xTIndexesPart2
    print(len(xTIndexes))
    xTIndexesPart1 = list(np.random.choice(xTIndexes, size=len(xTIndexes), replace=False))
    #xTIndexes = np.sort(xTIndexes)
    return(xTIndexes)


In [634]:
df['close_direction'].unique()

array([1, 0, 2])

In [635]:
#train_indexes = mystratifiedOutputSampler(df,'close_direction')

In [636]:
#onehot_encoded[0:30]
print(len(a))
print(len(AllWithAnyOtherThanNeutal), len(OnlyNeutal))

2782
1772 7044


In [637]:
#train_indexes = getBalancedSamples(x_train)

In [638]:
#y_train
train_indexes = getBalancedSamples(x_train)
X_train = x_train[train_indexes]
Y_train = y_train[train_indexes]

#X_train = x_train
#Y_train = y_train
Y_train

2
481
7598
337
573


array([[ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       ..., 
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.]])

In [640]:
model = Sequential()
# input_shape = number of time-steps, number-of-features
model.add(LSTM(128,input_shape=(window_size,len(input_cols)),
               activation='softsign', 
               inner_activation='hard_sigmoid', 
               return_sequences=True))
model.add(LSTM(128, activation='softsign', recurrent_activation='hard_sigmoid'))
#model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
#model.add(TimeDistributedDense(11))
#model.add(Dense(128))
model.add(Dense(128,activation='tanh'))
#model.add(Dropout(0.5))
model.add(Dense(64, activation='tanh'))
#model.add(Dropout(0.2))
model.add(Dense(32, activation='tanh'))
#model.add(Dropout(0.2))
model.add(Dense(output_dim=np.shape(y_train)[1], activation='softmax'))
#model.add(Activation('sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

/home/mygpu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, activation="softsign", return_sequences=True, input_shape=(1, 6), recurrent_activation="hard_sigmoid")`
/home/mygpu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=3)`


In [641]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_59 (LSTM)               (None, 1, 128)            69120     
_________________________________________________________________
lstm_60 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_50 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_104 (Dense)            (None, 128)               16512     
_________________________________________________________________
dense_105 (Dense)            (None, 64)                8256      
_________________________________________________________________
dense_106 (Dense)            (None, 32)                2080      
_________________________________________________________________
dense_107 (Dense)            (None, 3)                 99        
Total para

In [642]:
print('Train...')
model.fit(X_train, Y_train,
          batch_size=1,
          epochs=5,
          validation_data=(X_train, Y_train))
#score, acc = 


Train...
Train on 573 samples, validate on 573 samples
Epoch 1/5
573/573 [==============================] - 8s - loss: 1.0983 - val_loss: 1.0899
Epoch 2/5
573/573 [==============================] - 6s - loss: 1.0828 - val_loss: 1.0556
Epoch 3/5
573/573 [==============================] - 6s - loss: 1.0686 - val_loss: 1.0404
Epoch 4/5
573/573 [==============================] - 6s - loss: 1.0491 - val_loss: 0.9974
Epoch 5/5
573/573 [==============================] - 6s - loss: 1.0275 - val_loss: 0.9685


In [643]:
score = model.evaluate(X_train, Y_train,batch_size=1)
print()
print('Test score:', score)
print(model.predict(X_train))
#print('Test accuracy:', confusion_matrix(Y_train,model.predict_classes(X_train)))

558/573 [============================>.] - ETA: 0s
Test score: 0.968456945546
[[ 0.2302704   0.52216446  0.24756512]
 [ 0.21386819  0.62846506  0.15766674]
 [ 0.11151202  0.19101492  0.69747311]
 ..., 
 [ 0.20812508  0.60959494  0.18227994]
 [ 0.17636201  0.35002598  0.47361198]
 [ 0.21279116  0.44756529  0.33964354]]


In [644]:
#model.predict(x_train)
a = model.predict_classes(X_train)

573/573 [==============================] - 0s     


In [645]:
b = set(a)
a

array([1, 1, 2, 1, 1, 1, 1, 0, 0, 2, 0, 1, 1, 2, 0, 0, 2, 0, 1, 0, 1, 1, 1,
       2, 1, 1, 1, 0, 1, 2, 2, 1, 0, 0, 0, 1, 2, 0, 1, 1, 1, 2, 1, 2, 1, 0,
       2, 2, 2, 1, 0, 0, 1, 1, 0, 2, 2, 1, 0, 1, 1, 1, 2, 0, 0, 1, 1, 1, 1,
       2, 2, 1, 1, 2, 1, 2, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 0, 2,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 2, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 2, 1, 1, 0, 0, 2, 2, 0, 1, 1, 1, 1, 2, 1, 0, 1, 2, 1, 2, 1, 1, 1,
       1, 0, 2, 1, 2, 0, 1, 1, 1, 0, 2, 1, 2, 1, 1, 1, 1, 1, 0, 2, 1, 2, 2,
       1, 2, 1, 1, 1, 0, 1, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 0, 1,
       1, 1, 2, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 0, 2, 1, 1, 1, 1, 2, 1, 1, 0, 0, 0, 1,
       2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 0, 1, 2, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 2, 1, 0, 1, 1, 2, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 0,
       2, 2,

In [646]:
y_train
#list(y_train1)
#onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoder.active_features_
labels = Y_train.dot(onehot_encoder.active_features_).astype(int)

In [648]:
labels

array([2, 1, 2, 1, 2, 1, 1, 2, 0, 1, 0, 0, 2, 2, 0, 0, 2, 0, 2, 0, 1, 1, 1,
       2, 1, 0, 1, 0, 1, 2, 2, 2, 0, 1, 0, 0, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1,
       2, 2, 2, 1, 0, 0, 2, 2, 0, 2, 2, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       2, 2, 2, 2, 2, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 2, 1, 1, 1,
       2, 1, 1, 0, 0, 1, 2, 2, 1, 2, 1, 0, 2, 1, 0, 1, 1, 2, 0, 1, 1, 1, 0,
       0, 2, 1, 0, 0, 0, 2, 2, 0, 0, 2, 1, 0, 2, 2, 0, 1, 1, 2, 1, 1, 2, 1,
       1, 0, 2, 1, 2, 0, 1, 2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 0, 2, 1, 2, 1,
       1, 2, 2, 1, 2, 0, 1, 0, 0, 1, 0, 0, 2, 1, 2, 0, 2, 1, 1, 0, 2, 0, 1,
       1, 1, 2, 2, 1, 2, 0, 0, 1, 0, 1, 0, 2, 1, 1, 1, 0, 2, 0, 2, 1, 1, 2,
       0, 2, 1, 1, 0, 0, 2, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 2, 0, 0, 1, 1,
       0, 2, 2, 1, 0, 0, 2, 1, 1, 2, 0, 2, 2, 1, 1, 2, 0, 0, 1, 0, 0, 2, 0,
       1, 1, 1, 0, 1, 2, 1, 2, 0, 0, 1, 2, 0, 2, 2, 2, 1, 1, 2, 2, 1, 0, 1,
       1, 1, 0, 2, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 2, 0, 1, 1, 2, 2, 1, 1, 0,
       2, 2,

In [665]:
output = pd.DataFrame(columns=['predicted','actual'])
output['predicted'] = a.tolist()
output['actual'] = labels.tolist()
output['Stock'] = list(mydf['Stock_name'].iloc[train_indexes])
output['Sentiment'] = list(mydf['sentiment'].iloc[train_indexes])

In [666]:
output

,predicted,actual,Stock,Sentiment
0,1,2,APPLE,3
1,1,1,IBM,3
2,2,2,GE,4
3,1,1,IBM,3
4,1,2,BOEING,3
5,1,1,MCD,3
6,1,1,APPLE,4
7,0,2,MCD,3
8,0,0,APPLE,3
9,2,1,MICROSOFT,3


In [664]:
#mydf.iloc[train_indexes]

In [ ]:
ALL

In [ ]:
a=[[[]]]

In [ ]:
a[0] = ALL[0:5]

In [ ]:
a.append(ALL[1:6])

In [ ]:
a

In [ ]:
np.array(a)

In [ ]:
len(ALL)